<h1>Welcome!</h1>
<h3>Thanks for checking out my Capstone Project for the IBM Data Science Certificate</h3>
<p>In this notebook I will be writing the code to solve the problem laid out in week one and hopefully coming up with some answers!
<br>--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
#Only run this cell if the imports don't work
#!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes 
#!conda install BeautifulSoup4 --yes

In [7]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from pandas.io.json import json_normalize
import requests
import re
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
print('libraries imported!')

libraries imported!


<h3>Here I will get the neighborhoods of Houston off of the wikipedia list I found online using BeautifulSoup, clean it up, turn it into a dataframe, and remove the columns which are not useful to the eventual analysis.

In [8]:
webpage = requests.get('https://en.wikipedia.org/wiki/List_of_Houston_neighborhoods')
print(webpage)
#Create the BeautifulSoup object
soup = BeautifulSoup(webpage.content,'html.parser')
#Get the table
soup_table = soup.find('table')
#Turn the table into a string
textonly = soup_table.get_text(',')
#Turn the string into a list
full_list = textonly.split('\n')
#Clean the list
clean_list = []
for i in full_list:
    if i != ',' and i != '':
        i = i.replace(',','')
        clean_list.append(i)
columns = clean_list[0:3]
indices = []
for i in range(4, len(clean_list), 4):
    indices.append(clean_list[i])
neighborhoods = []
for i in range(5, len(clean_list), 4):
    neighborhoods.append(clean_list[i])
relative_location = []
for i in range(6, len(clean_list), 4):
    relative_location.append(clean_list[i])
boundaries = []
for i in range(7, len(clean_list), 4):
    boundaries.append(clean_list[i])
data = {'Index':indices, 'Neighborhood':neighborhoods, 'RelativeLocation':relative_location, 'Boundaries':boundaries}
neighborhood_df = pd.DataFrame(data)
neighborhood_df.drop(neighborhood_df[['Index', 'RelativeLocation', 'Boundaries']], axis=1, inplace=True)
neighborhood_df.loc[11, 'Neighborhood'] = 'Central Northwest'
neighborhood_df.loc[19, 'Neighborhood'] = 'Mid-west'
neighborhood_df.loc[35, 'Neighborhood'] = 'Brays Oaks'
neighborhood_df.loc[65, 'Neighborhood'] = 'Museum Park'
print(neighborhood_df.shape)
neighborhood_df.head()

<Response [200]>
(88, 1)


,Neighborhood
0,Willowbrook
1,Greater Greenspoint
2,Carverdale
3,Fairbanks / Northwest Crossing
4,Greater Inwood


<h3>Next, I will use geopy to get the latitudes and longitudes of each neighborhood.

In [9]:
all_lats_longs = []
for neighborhood in neighborhood_df.Neighborhood:
    try:
        g = Nominatim(user_agent='houston_explorer', timeout=5)
        location = g.geocode(neighborhood + ', Houston, Texas')
        latitude = location.latitude
        longitude = location.longitude
        all_lats_longs.append([neighborhood, latitude, longitude])
        print('added '+neighborhood)
    except AttributeError:
        all_lats_longs.append([neighborhood, '', ''])
        print('added '+neighborhood+', but empty')


added Willowbrook
added Greater Greenspoint
added Carverdale
added Fairbanks / Northwest Crossing
added Greater Inwood, but empty
added Acres Home
added Hidden Valley
added Westbranch
added Addicks / Park Ten
added Spring Branch West
added Langwood
added Central Northwest
added Independence Heights
added Lazybrook / Timbergrove, but empty
added Greater Heights
added Memorial
added Eldridge / West Oaks, but empty
added Briar Forest
added Westchase
added Mid-west
added Greater Uptown, but empty
added Washington Avenue Coalition / Memorial Park, but empty
added Afton Oaks / River Oaks, but empty
added Neartown / Montrose, but empty
added Alief
added Sharpstown
added Gulfton
added University Place
added Westwood
added Braeburn
added Meyerland
added Braeswood
added Medical Center
added Astrodome Area
added South Main
added Brays Oaks
added Westbury
added Willow Meadows / Willowbend, but empty
added Fondren Gardens
added Central Southwest
added Fort Bend / Houston
added IAH Airport
added Kin

<h3>Now I will take my list of lists and turn it into a dictionary to then make into my final locations dataframe.

In [10]:
#create lats and longs lists
lats = []
longs = []
for i in all_lats_longs:
    lats.append(i[1])
for i in all_lats_longs:
    longs.append(i[2])
#create and add columns to locations_df
locations_df = neighborhood_df
locations_df['Latitude'] = lats
locations_df['Longitude'] = longs
#Fill in empty spaces with hand-googled data. If the dataset were much larger, then I would probaby just drop 
#these rows... however, with only 88 rows, every row counts.
locations_df.loc[4, 'Latitude'] = 29.8657
locations_df.loc[4, 'Longitude'] = -95.4804
locations_df.loc[13, 'Latitude'] = 29.8016
locations_df.loc[13, 'Longitude'] = -95.4381
locations_df.loc[16, 'Latitude'] = 29.7322
locations_df.loc[16, 'Longitude'] = -95.6483
locations_df.loc[20, 'Latitude'] = 29.741
locations_df.loc[20, 'Longitude'] = -95.461
locations_df.loc[21, 'Latitude'] = 29.7771
locations_df.loc[21, 'Longitude'] = -95.4355
locations_df.loc[22, 'Latitude'] = 29.7369
locations_df.loc[22, 'Longitude'] = -95.4532
locations_df.loc[23, 'Latitude'] = 29.7427
locations_df.loc[23, 'Longitude'] = -95.3992
locations_df.loc[37, 'Latitude'] = 29.6733
locations_df.loc[37, 'Longitude'] = -95.4399
locations_df.loc[44, 'Latitude'] = 29.8595
locations_df.loc[44, 'Longitude'] = -95.3847
locations_df.loc[45, 'Latitude'] = 29.8482
locations_df.loc[45, 'Longitude'] = -95.3429
locations_df.loc[46, 'Latitude'] = 29.8714
locations_df.loc[46, 'Longitude'] = -95.3006
locations_df.loc[52, 'Latitude'] = 29.8052
locations_df.loc[52, 'Longitude'] = -95.2405
locations_df.loc[53, 'Latitude'] = 29.8165
locations_df.loc[53, 'Longitude'] = -95.2136
locations_df.loc[54, 'Latitude'] = 29.7761
locations_df.loc[54, 'Longitude'] = -95.3265
locations_df.loc[55, 'Latitude'] = 29.7744
locations_df.loc[55, 'Longitude'] = -95.3011
locations_df.loc[58, 'Latitude'] = 29.7437
locations_df.loc[58, 'Longitude'] = -95.2554
locations_df.loc[59, 'Latitude'] = 29.7579
locations_df.loc[59, 'Longitude'] = -95.3809
locations_df.loc[63, 'Latitude'] = 29.7357
locations_df.loc[63, 'Longitude'] = -95.3340
locations_df.loc[65, 'Latitude'] = 29.721
locations_df.loc[65, 'Longitude'] = -95.3858
locations_df.loc[66, 'Latitude'] = 29.7278
locations_df.loc[66, 'Longitude'] = -95.3499
locations_df.loc[67, 'Latitude'] = 29.6912
locations_df.loc[67, 'Longitude'] = -95.3529
locations_df.loc[68, 'Latitude'] = 29.7086
locations_df.loc[68, 'Longitude'] = -95.3168
locations_df.loc[72, 'Latitude'] = 29.6852
locations_df.loc[72, 'Longitude'] = -95.2963
locations_df.loc[75, 'Latitude'] = 29.6276
locations_df.loc[75, 'Longitude'] = -95.3556
locations_df.loc[77, 'Latitude'] = 29.6254
locations_df.loc[77, 'Longitude'] = -95.2854
locations_df.loc[79, 'Latitude'] = 29.6015
locations_df.loc[79, 'Longitude'] = -95.2161
locations_df.loc[86, 'Latitude'] = 29.7327
locations_df.loc[86, 'Longitude'] = -95.4332

#These two were assigned incorrectly by Nominatim so I am adjusting them by hand.
locations_df.loc[69, 'Latitude'] = 29.7065
locations_df.loc[69, 'Longitude'] = -95.2861
locations_df.loc[80, 'Latitude'] = 29.5857
locations_df.loc[80, 'Longitude'] = -95.1328

#Uncomment only if changes are made to the csv and you want to update the file
#locations_df.to_csv('HoustonSNsLatLong.csv')

print(locations_df.shape)
locations_df

(88, 3)


,Neighborhood,Latitude,Longitude
0,Willowbrook,29.6603,-95.4561
1,Greater Greenspoint,29.9447,-95.4161
2,Carverdale,29.8487,-95.5395
3,Fairbanks / Northwest Crossing,29.8527,-95.5244
4,Greater Inwood,29.8657,-95.4804
5,Acres Home,29.7708,-95.3281
6,Hidden Valley,30.0538,-95.1194
7,Westbranch,29.8395,-95.5515
8,Addicks / Park Ten,29.7825,-95.6424
9,Spring Branch West,29.8014,-95.5479


<h3>Next I will make a map to see where all of the neighborhoods are in the Houston area and to make sure that the lats and longs worked.

In [11]:
address = 'Houston, Texas'
geolocator = Nominatim(user_agent="houston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Houston are {}, {}.'.format(latitude, longitude))

map_houston = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(locations_df['Latitude'], locations_df['Longitude'], locations_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston) 
map_houston

#Pecan Park and Clearlake were both wrong from Nominatim, so I will go back up and fix them manually.

The geograpical coordinates of Houston are 29.7589382, -95.3676974.


<h3>Next, I will pass my credentials and use the Foursquare API to get venues in each of the 88 neighborhoods. I will also create a function to save those venues and their categories in order to use them later for my clustering model. Since Houston is a big city, I will be using a distance of 750 meters for the radius since the neighborhoods are more spread out than usual.

In [12]:
# @hidden_cell
CLIENT_ID = '###'
CLIENT_SECRET = '###'
VERSION = '###'

print('FourSquare credentials loaded!')

FourSquare credentials loaded!


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'NeighborhoodLatitude', 
                  'NeighborhoodLongitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    return(nearby_venues)
print('Function made!')

Function made!


In [14]:
houston_venues = getNearbyVenues(names=locations_df['Neighborhood'],
                                   latitudes=locations_df['Latitude'],
                                   longitudes=locations_df['Longitude']
                                  )

Willowbrook
Greater Greenspoint
Carverdale
Fairbanks / Northwest Crossing
Greater Inwood
Acres Home
Hidden Valley
Westbranch
Addicks / Park Ten
Spring Branch West
Langwood
Central Northwest
Independence Heights
Lazybrook / Timbergrove
Greater Heights
Memorial
Eldridge / West Oaks
Briar Forest
Westchase
Mid-west
Greater Uptown
Washington Avenue Coalition / Memorial Park
Afton Oaks / River Oaks
Neartown / Montrose
Alief
Sharpstown
Gulfton
University Place
Westwood
Braeburn
Meyerland
Braeswood
Medical Center
Astrodome Area
South Main
Brays Oaks
Westbury
Willow Meadows / Willowbend
Fondren Gardens
Central Southwest
Fort Bend / Houston
IAH Airport
Kingwood
Lake Houston
Northside / Northline
Eastex - Jensen
East Little York / Homestead
Trinity / Houston Gardens
East Houston
Settegast
Northside Village
Kashmere Gardens
El Dorado / Oates Prairie
Hunterwood
Greater Fifth Ward
Denver Harbor / Port Houston
Pleasantville Area
Northshore
Clinton Park / Tri-Community
Fourth Ward
Downtown
Midtown
Sec

In [15]:
print(houston_venues.shape)
print('There are {} unique categories.'.format(len(houston_venues.VenueCategory.unique())))
print('There are {} unique neighborhoods.'.format(len(houston_venues.Neighborhood.unique())))

venue_count = houston_venues.groupby('Neighborhood').Venue.count().to_frame().reset_index()
#Only uncomment if you make a change to the dataframe and want to update the csv file
#venue_count.to_csv('VenueCountPerNeighborhood.csv')

print(houston_venues.groupby('Neighborhood').Venue.count())
houston_venues.head()

(2612, 7)
There are 282 unique categories.
There are 87 unique neighborhoods.
Neighborhood
Acres Home                                      10
Addicks / Park Ten                              43
Afton Oaks / River Oaks                        100
Alief                                           26
Astrodome Area                                  44
Braeburn                                        33
Braeswood                                       12
Brays Oaks                                       9
Briar Forest                                    23
Carverdale                                       9
Central Northwest                                8
Central Southwest                              100
Clear Lake                                       4
Clinton Park / Tri-Community                     4
Denver Harbor / Port Houston                    19
Downtown                                       100
East Houston                                     3
East Little York / Homestead              

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,Willowbrook,29.660254,-95.456096,Kolache Factory,29.666938,-95.462662,Breakfast Spot
1,Willowbrook,29.660254,-95.456096,Emmit's Place,29.657188,-95.463080,Bar
2,Willowbrook,29.660254,-95.456096,Popeyes Louisiana Kitchen,29.664607,-95.463331,Fried Chicken Joint
3,Willowbrook,29.660254,-95.456096,Annie's Burgers,29.663315,-95.463785,Burger Joint
4,Willowbrook,29.660254,-95.456096,Hunter's Pub,29.666188,-95.462762,Dive Bar


<h3>Here, I noticed that after running my foursquare function I was only getting 86 neighborhoods but the original dataframe that I made had 88. I used this next cell to figure out which neighborhoods weren't working and then looked at the map to see where they were. It turns out that Nominatim got the wrong coordinates and put them miles away from Houston, so they had no data. I then searched for their coordinates on google and manually changed them above, so now I am commenting out this section because it is not as useful. It does, however, tell me if one neighborhood returns 0 venues.

In [16]:
after_4s = houston_venues.Neighborhood.unique()
before_4s = locations_df.Neighborhood.unique()
print(len(after_4s))
print(len(before_4s))
not_in = []
for i in before_4s:
    if i not in after_4s:
        not_in.append(i)
not_in

87
88


['Hunterwood']

<h3>Next I will begin looking at top venue categories for different neighborhoods in order to rank the top ten most common in each distinct neighborhood. The first step in this is one-hot encoding and getting dummies.

In [17]:
houston_onehot = pd.get_dummies(houston_venues[['VenueCategory']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
houston_onehot['Neighborhood'] = houston_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [houston_onehot.columns[-1]] + list(houston_onehot.columns[:-1])
houston_onehot = houston_onehot[fixed_columns]

houston_onehot.head()

,Neighborhood,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Residence Hall,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vi

In [18]:
houston_grouped = houston_onehot.groupby('Neighborhood').mean().reset_index()
print(houston_grouped.shape)
houston_grouped.head()

(87, 283)


,Neighborhood,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Residence Hall,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vi

<h3>After one-hot, now I need to get the top ten categories for each neighborhood and save it into a new dataframe.

In [19]:
num_top_cats = 10
#for hood in houston_grouped['Neighborhood']:
#    print("----"+hood+"----")
#    temp = houston_grouped[houston_grouped['Neighborhood'] == hood].T.reset_index()
#    temp.columns = ['category','freq']
#    temp = temp.iloc[1:]
#    temp['freq'] = temp['freq'].astype(float)
#    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_cats))
#    print('\n')

def return_most_common_venues(row, num_top_cats):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_cats]

indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_cats):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = houston_grouped['Neighborhood']

for ind in np.arange(houston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_grouped.iloc[ind, :], num_top_cats)

#Only uncomment if a change is made to the dataframe and you want to update it.
#neighborhoods_venues_sorted.to_csv('top_10_per_hood.csv')
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acres Home,Park,Pub,Intersection,Arts & Entertainment,Business Service,Music Venue,Shipping Store,Market,Fried Chicken Joint,Fast Food Restaurant
1,Addicks / Park Ten,Hotel,Coffee Shop,Mexican Restaurant,Sandwich Place,Indian Restaurant,Ice Cream Shop,Donut Shop,Bakery,Bank,Bar
2,Afton Oaks / River Oaks,Women's Store,Furniture / Home Store,Coffee Shop,French Restaurant,American Restaurant,Cosmetics Shop,Cajun / Creole Restaurant,Rental Car Location,Boutique,Clothing Store
3,Alief,Vietnamese Restaurant,Bubble Tea Shop,Café,Football Stadium,Restaurant,Massage Studio,Gaming Cafe,Gas Station,Bakery,Latin American Restaurant
4,Astrodome Area,Football Stadium,Rental Car Location,Coffee Shop,Stadium,Bar,Gas Station,General Entertainment,BBQ Joint,Pharmacy,Gym / Fitness Center


<h3>Now that we have the one-hot dataframe and a more readable dataframe with the top 10 categories, it is time to build and train the KMeans model and get some labels for our neighborhoods.

In [20]:
k = 5
houston_grouped_cluster = houston_grouped.drop('Neighborhood', axis=1)
model = KMeans(n_clusters=k, n_init=100, max_iter=50)
model.fit(houston_grouped_cluster)
model.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

<h3>Well, that wasn't very useful. Let's try Agglomertive clustering.

In [21]:
from scipy.cluster import hierarchy
from scipy.spatial import distance_matrix
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt

agglom = AgglomerativeClustering(n_clusters=9, affinity='euclidean', linkage='ward')
agglom.fit(houston_grouped_cluster)
print(agglom.labels_)
x_min, x_max = np.min(houston_grouped_cluster, axis=0), np.max(houston_grouped_cluster, axis=0)
x = (houston_grouped_cluster - x_min)/(x_max-x_min)
dist_matrix = distance_matrix(x,x)
plt.figure(figsize=[20,10])
z = hierarchy.linkage(dist_matrix, 'ward')
dendro = hierarchy.dendrogram(z, leaf_font_size=12)
plt.show()

[2 0 0 0 0 0 0 0 0 0 3 0 0 2 0 0 8 1 0 0 6 0 0 2 2 0 0 0 2 0 0 0 0 0 0 0 0
 0 0 2 1 0 0 0 0 5 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 0 2 0 0 0 3 4 0
 0 0 0 0 0 0 0 2 0 0 0 0 0]


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:13: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  del sys.path[0]


<Figure size 2000x1000 with 1 Axes>

<h3>Well, the dendrogram worked but for some reason it didn't pay attention to my n_clusters so I just made the dendrogram bigger and will be putting it into clusters manually based on where the dendrogram split 9 times. and then sort them into one list of ClusterLabels in order to add it to my dataframe

In [22]:
cluster_0_idx = [35, 2, 59]
cluster_1_idx = [58, 53]
cluster_2_idx = [52, 11, 77]
cluster_3_idx = [25, 4, 3, 83, 36, 78, 15]
cluster_4_idx = [81, 48, 34, 54, 21, 41, 66, 56, 42, 8, 64, 6, 84, 38]
cluster_5_idx = [85, 22, 82, 80, 60, 9, 23, 86, 27, 68, 79, 49, 70, 63, 5, 50, 61, 19, 46, 1, 44, 29, 28, 0]
cluster_6_idx = [74, 73, 75]
cluster_7_idx = [55, 33, 12, 69, 57, 67, 39, 47, 13, 76, 62, 51, 32, 18, 14, 7, 30, 24, 65, 16, 43, 71, 10]
cluster_8_idx = [72, 20, 17, 45, 40, 37, 26, 31]
all_clusters = [cluster_0_idx,cluster_1_idx,cluster_2_idx,cluster_3_idx,cluster_4_idx,cluster_5_idx,cluster_6_idx,cluster_7_idx,cluster_8_idx]
all_clusters

[[35, 2, 59],
 [58, 53],
 [52, 11, 77],
 [25, 4, 3, 83, 36, 78, 15],
 [81, 48, 34, 54, 21, 41, 66, 56, 42, 8, 64, 6, 84, 38],
 [85,
  22,
  82,
  80,
  60,
  9,
  23,
  86,
  27,
  68,
  79,
  49,
  70,
  63,
  5,
  50,
  61,
  19,
  46,
  1,
  44,
  29,
  28,
  0],
 [74, 73, 75],
 [55,
  33,
  12,
  69,
  57,
  67,
  39,
  47,
  13,
  76,
  62,
  51,
  32,
  18,
  14,
  7,
  30,
  24,
  65,
  16,
  43,
  71,
  10],
 [72, 20, 17, 45, 40, 37, 26, 31]]

In [24]:
cluster_labels = []
for num in range(0, 87):
    for lst in all_clusters:
        for i in lst:
            if i == num:
                cluster_labels.append(all_clusters.index(lst))
cluster_labels

[5,
 5,
 0,
 3,
 3,
 5,
 4,
 7,
 4,
 5,
 7,
 2,
 7,
 7,
 7,
 3,
 7,
 8,
 7,
 5,
 8,
 4,
 5,
 5,
 7,
 3,
 8,
 5,
 5,
 5,
 7,
 8,
 7,
 7,
 4,
 0,
 3,
 8,
 4,
 7,
 8,
 4,
 4,
 7,
 5,
 8,
 5,
 7,
 4,
 5,
 5,
 7,
 2,
 1,
 4,
 7,
 4,
 7,
 1,
 0,
 5,
 5,
 7,
 5,
 4,
 7,
 4,
 7,
 5,
 7,
 5,
 7,
 8,
 6,
 6,
 6,
 7,
 2,
 3,
 5,
 5,
 4,
 5,
 3,
 4,
 5,
 5]

<h3>Now I will add the labels to my houston_grouped_cluster dataframe in order to run a KNN classification model. Then I will split the data and run the KNN model to see how it fares.

In [26]:
houston_grouped_cluster['ClusterLabel'] = cluster_labels
houston_grouped_cluster.head()

#After running the KNN model, I think it best to remove the clusters with less than 5 neighborhoods. Let's try it.
#small = cluster_0_idx
#small.extend(cluster_1_idx)
#small.extend(cluster_2_idx)
#small.extend(cluster_6_idx)
#small = set(small)
#small = sorted(small, reverse=True)
#for i in small:
#    houston_grouped_cluster.drop(index=i, axis=0, inplace=True)

print(houston_grouped_cluster.shape)
houston_grouped_cluster.head()

(87, 283)


,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Residence Hall,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vie

In [38]:
houston_grouped['ClusterLabel'] = cluster_labels
for_cluster_map = pd.merge(locations_df, houston_grouped,
                          left_on='Neighborhood',
                          right_on='Neighborhood',
                          suffixes=['_group', '_loc'])
print(for_cluster_map.shape)
for_cluster_map.head()

(87, 286)


,Neighborhood,Latitude,Longitude,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Casino,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Residence Hall,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Street Food Gathering,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Veterinarian,

In [39]:
kclusters=9
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(for_cluster_map['Latitude'], for_cluster_map['Longitude'], for_cluster_map['Neighborhood'], for_cluster_map['ClusterLabel']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

y = houston_grouped_cluster['ClusterLabel']
x = houston_grouped_cluster.drop(['ClusterLabel'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.15)
#find best k
#scores = []
#for i in range(1,11):
#    k = i
#    classifier = KNeighborsClassifier(n_neighbors=k, weights='distance')
#    classifier.fit(x_train, y_train)
#    score = classifier.score(x_test, y_test)
#    scores.append(score)
#print(scores)
#plt.plot(scores)

best_k = 5
classifier = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
classifier.fit(x_train, y_train)
classifier.score(x_test, y_test)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0.32432432432432434

<h3>Let's examine each cluster!

In [50]:
neighborhoods_venues_sorted['Cluster'] = cluster_labels
neighborhoods_venues_sorted.head()

print('Cluster 0')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 0, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 0


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
2,Afton Oaks / River Oaks,Women's Store,Furniture / Home Store,Coffee Shop,French Restaurant,American Restaurant,Cosmetics Shop,Cajun / Creole Restaurant,Rental Car Location,Boutique,Clothing Store,0
35,Greater Uptown,Clothing Store,Department Store,Hotel,American Restaurant,Italian Restaurant,Ice Cream Shop,Women's Store,Sporting Goods Shop,Steakhouse,Boutique,0
59,Neartown / Montrose,Italian Restaurant,Bar,Coffee Shop,Cocktail Bar,New American Restaurant,Thrift / Vintage Store,Burger Joint,Pizza Place,Art Gallery,Mediterranean Restaurant,0


In [51]:
print('Cluster 1')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 1, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 1


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
53,Memorial,Zoo Exhibit,Science Museum,Food Truck,Garden,Trail,Sculpture Garden,Café,Art Museum,Art Gallery,American Restaurant,1
58,Museum Park,Zoo Exhibit,Food Truck,Science Museum,Café,Art Museum,Garden,Coffee Shop,Sculpture Garden,Southern / Soul Food Restaurant,Trail,1


In [52]:
print('Cluster 2')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 2, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 2


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
11,Central Southwest,Hotel,Mexican Restaurant,Coffee Shop,Burger Joint,Bar,Southern / Soul Food Restaurant,Theater,Sandwich Place,Pizza Place,Italian Restaurant,2
52,Medical Center,Hotel,Mexican Restaurant,Coffee Shop,Burger Joint,Bar,Southern / Soul Food Restaurant,Theater,Sandwich Place,Pizza Place,Italian Restaurant,2
77,Sunnyside,Hotel,Mexican Restaurant,Coffee Shop,Burger Joint,Bar,Southern / Soul Food Restaurant,Theater,Sandwich Place,Pizza Place,Italian Restaurant,2


In [53]:
print('Cluster 3')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 3, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 3


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
3,Alief,Vietnamese Restaurant,Bubble Tea Shop,Café,Football Stadium,Restaurant,Massage Studio,Gaming Cafe,Gas Station,Bakery,Latin American Restaurant,3
4,Astrodome Area,Football Stadium,Rental Car Location,Coffee Shop,Stadium,Bar,Gas Station,General Entertainment,BBQ Joint,Pharmacy,Gym / Fitness Center,3
15,Downtown,Burger Joint,Hotel,Bar,Cocktail Bar,Coffee Shop,Southern / Soul Food Restaurant,Park,Concert Hall,Pizza Place,Theater,3
25,Fourth Ward,Park,Bar,American Restaurant,Hotel,Sandwich Place,Coffee Shop,Trail,Pub,Gym,Restaurant,3
36,Greenway / Upper Kirby,Italian Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Burger Joint,Bakery,Spa,Café,Chinese Restaurant,Thai Restaurant,3
78,Trinity / Houston Gardens,Park,Food Truck,Bar,Trail,Coffee Shop,Theater,Art Gallery,Brewery,Gym,Dance Studio,3
83,Westchase,Breakfast Spot,Pharmacy,Taco Place,Grocery Store,Discount Store,Sandwich Place,Sushi Restaurant,Steakhouse,Big Box Store,Bike Shop,3


In [54]:
print('Cluster 4')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 4, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 4


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
6,Braeswood,Sandwich Place,Pizza Place,Thrift / Vintage Store,Department Store,Discount Store,Gym,Supplement Shop,Fast Food Restaurant,Lawyer,Fried Chicken Joint,4
8,Briar Forest,Convenience Store,Sandwich Place,Mexican Restaurant,Pizza Place,Cajun / Creole Restaurant,Chinese Restaurant,Salad Place,Coworking Space,Pool,Donut Shop,4
21,Eldridge / West Oaks,Sandwich Place,Fast Food Restaurant,Donut Shop,Clothing Store,Liquor Store,Indian Restaurant,Mobile Phone Shop,Movie Theater,Sporting Goods Shop,Gas Station,4
34,Greater Third Ward,Fast Food Restaurant,Fried Chicken Joint,College Baseball Diamond,Taco Place,Basketball Stadium,Student Center,Bagel Shop,Donut Shop,Seafood Restaurant,Opera House,4
38,Gulfton,Mexican Restaurant,Mobile Phone Shop,Fast Food Restaurant,Bakery,Salvadoran Restaurant,Clothing Store,Latin American Restaurant,Bus Station,Bank,Bar,4
41,IAH Airport,Coffee Shop,Sandwich Place,Airport Lounge,Airport Terminal,Snack Place,Airport,Airport Service,Japanese Restaurant,Seafood Restaurant,Accessories Store,4
42,Independence Heights,Mobile Phone Shop,Shoe Store,Discount Store,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Breakfast Spot,Women's Store,Supplement Shop,Burger Joint,4
48,Lazybrook / Timbergrove,Gym,Italian Restaurant,Park,Pizza Place,Sandwich Place,Restaurant,Thai Restaurant,Diner,Coffee Shop,New American Restaurant,4
54,Meyerland,Department Store,Furniture / Home Store,Cosmetics Shop,Café,Accessories Store,Big Box Store,Clothing Store,Bank,Basketball Court,Sandwich Place,4
56,Midtown,Nightclub,Gym / Fitness Center,Bar,Burger Joint,Food Truck,Brewery,Fast Food Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Music Venue,4


In [55]:
print('Cluster 5')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 5, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 5


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Acres Home,Park,Pub,Intersection,Arts & Entertainment,Business Service,Music Venue,Shipping Store,Market,Fried Chicken Joint,Fast Food Restaurant,5
1,Addicks / Park Ten,Hotel,Coffee Shop,Mexican Restaurant,Sandwich Place,Indian Restaurant,Ice Cream Shop,Donut Shop,Bakery,Bank,Bar,5
5,Braeburn,Pharmacy,Discount Store,Food,Bar,Sandwich Place,Clothing Store,Hotel,Mobile Phone Shop,Chinese Restaurant,Bank,5
9,Carverdale,Gas Station,Burger Joint,Taco Place,BBQ Joint,Breakfast Spot,Locksmith,Supermarket,Falafel Restaurant,Farmers Market,Fast Food Restaurant,5
19,Edgebrook,Fast Food Restaurant,Seafood Restaurant,Bank,Shoe Store,Discount Store,Video Store,Rental Car Location,Gas Station,Sandwich Place,Check Cashing Service,5
22,Fairbanks / Northwest Crossing,Wings Joint,Mexican Restaurant,Motel,Bakery,BBQ Joint,Gas Station,Sandwich Place,Athletics & Sports,Café,Asian Restaurant,5
23,Fondren Gardens,Video Store,Construction & Landscaping,Park,Burger Joint,Gym / Fitness Center,Pharmacy,Sandwich Place,Dry Cleaner,Shipping Store,Fried Chicken Joint,5
27,Greater Eastwood,Mexican Restaurant,Discount Store,Video Store,Food Truck,Bus Station,Café,Community Center,Pharmacy,Department Store,Park,5
28,Greater Fifth Ward,Park,Fried Chicken Joint,Fast Food Restaurant,Bank,Arts & Entertainment,Business Service,Market,Discount Store,Music Venue,Pharmacy,5
29,Greater Greenspoint,Fast Food Restaurant,Shoe Store,Sandwich Place,Bank,American Restaurant,Movie Theater,Supplement Shop,Chinese Restaurant,Pizza Place,Department Store,5


In [56]:
print('Cluster 6')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 6, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 6


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
73,Spring Branch Central,Food,Pizza Place,Grocery Store,Donut Shop,Bakery,Tapas Restaurant,Liquor Store,Fast Food Restaurant,Athletics & Sports,Shipping Store,6
74,Spring Branch East,Food,Pizza Place,Grocery Store,Donut Shop,Bakery,Tapas Restaurant,Liquor Store,Fast Food Restaurant,Athletics & Sports,Shipping Store,6
75,Spring Branch North,Food,Pizza Place,Grocery Store,Donut Shop,Bakery,Tapas Restaurant,Liquor Store,Fast Food Restaurant,Athletics & Sports,Shipping Store,6


In [57]:
print('Cluster 7')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 7, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 7


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
7,Brays Oaks,Mexican Restaurant,Gym / Fitness Center,Pizza Place,Park,Fast Food Restaurant,Baseball Field,Hardware Store,Taco Place,Zoo Exhibit,Empanada Restaurant,7
10,Central Northwest,Historic Site,Boat or Ferry,Harbor / Marina,Lounge,Zoo Exhibit,Farmers Market,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,7
12,Clear Lake,Lounge,Outdoors & Recreation,Spa,Salon / Barbershop,Zoo Exhibit,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,7
13,Clinton Park / Tri-Community,Theater,Trail,Recreation Center,Park,Zoo Exhibit,Eye Doctor,Duty-free Shop,Electronics Store,Empanada Restaurant,Exhibit,7
14,Denver Harbor / Port Houston,Mexican Restaurant,Fried Chicken Joint,Fast Food Restaurant,American Restaurant,Nail Salon,Latin American Restaurant,Skate Park,Pizza Place,Gas Station,Taco Place,7
16,East Houston,Pharmacy,Concert Hall,Museum,Financial or Legal Service,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,7
18,Eastex - Jensen,Fast Food Restaurant,Pizza Place,Gas Station,Discount Store,Sandwich Place,Fried Chicken Joint,Bank,Airport,Pharmacy,Grocery Store,7
24,Fort Bend / Houston,Pharmacy,Discount Store,Video Store,Donut Shop,Grocery Store,Park,Gas Station,Fast Food Restaurant,Optical Shop,Fried Chicken Joint,7
30,Greater Heights,Fried Chicken Joint,Diner,Mexican Restaurant,Bus Station,Fast Food Restaurant,BBQ Joint,Grocery Store,Seafood Restaurant,Comfort Food Restaurant,Convenience Store,7
32,Greater Inwood,Fast Food Restaurant,Discount Store,Video Store,Grocery Store,Seafood Restaurant,Pizza Place,Pharmacy,Cajun / Creole Restaurant,Sandwich Place,Chinese Restaurant,7


In [58]:
print('Cluster 8')
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Cluster'] == 8, neighborhoods_venues_sorted.columns[list(range(0, neighborhoods_venues_sorted.shape[1]))]]

Cluster 8


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
17,East Little York / Homestead,Discount Store,Grocery Store,Print Shop,Farmers Market,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Fast Food Restaurant,8
20,El Dorado / Oates Prairie,Shipping Store,Construction & Landscaping,Rental Car Location,Farmers Market,Duty-free Shop,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,8
26,Golfcrest / Bellfort / Reveille,Seafood Restaurant,Fast Food Restaurant,Mexican Restaurant,Construction & Landscaping,Breakfast Spot,Pizza Place,Pharmacy,Bakery,Video Store,Dog Run,8
31,Greater Hobby Area,Food Truck,Fast Food Restaurant,Gas Station,Construction & Landscaping,Grocery Store,Mexican Restaurant,Sandwich Place,Burger Joint,Farmers Market,Exhibit,8
37,Gulfgate Riverview / Pine Valley,Fast Food Restaurant,Food,Mexican Restaurant,Discount Store,Optical Shop,Mobile Phone Shop,Food Truck,Snack Place,Furniture / Home Store,Smoke Shop,8
40,Hidden Valley,Discount Store,Zoo Exhibit,Duty-free Shop,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,8
45,Lake Houston,Construction & Landscaping,Zoo Exhibit,Fast Food Restaurant,Electronics Store,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Financial or Legal Service,8
72,South Park,Airport Terminal,Coffee Shop,Zoo Exhibit,Financial or Legal Service,Empanada Restaurant,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,8
